Escape Pods
===========

You've blown up the LAMBCHOP doomsday device and broken the bunnies out of Lambda's prison - and now you need to escape from the space station as quickly and as orderly as possible! The bunnies have all gathered in various locations throughout the station, and need to make their way towards the seemingly endless amount of escape pods positioned in other parts of the station. You need to get the numerous bunnies through the various rooms to the escape pods. Unfortunately, the corridors between the rooms can only fit so many bunnies at a time. What's more, many of the corridors were resized to accommodate the LAMBCHOP, so they vary in how many bunnies can move through them at a time. 

Given the starting room numbers of the groups of bunnies, the room numbers of the escape pods, and how many bunnies can fit through at a time in each direction of every corridor in between, figure out how many bunnies can safely make it to the escape pods at a time at peak.

Write a function solution(entrances, exits, path) that takes an array of integers denoting where the groups of gathered bunnies are, an array of integers denoting where the escape pods are located, and an array of an array of integers of the corridors, returning the total number of bunnies that can get through at each time step as an int. The entrances and exits are disjoint and thus will never overlap. The path element path[A][B] = C describes that the corridor going from A to B can fit C bunnies at each time step.  There are at most 50 rooms connected by the corridors and at most 2000000 bunnies that will fit at a time.

For example, if you have:
entrances = [0, 1]
exits = [4, 5]
path = [

  [0, 0, 4, 6, 0, 0],  # Room 0: Bunnies
  
  [0, 0, 5, 2, 0, 0],  # Room 1: Bunnies
  
  [0, 0, 0, 0, 4, 4],  # Room 2: Intermediate room
  
  [0, 0, 0, 0, 6, 6],  # Room 3: Intermediate room
  
  [0, 0, 0, 0, 0, 0],  # Room 4: Escape pods
  
  [0, 0, 0, 0, 0, 0]] # Room 5: Escape pods


Then in each time step, the following might happen:

0 sends 4/4 bunnies to 2 and 6/6 bunnies to 3

1 sends 4/5 bunnies to 2 and 2/2 bunnies to 3

2 sends 4/4 bunnies to 4 and 4/4 bunnies to 5

3 sends 4/6 bunnies to 4 and 4/6 bunnies to 5

So, in total, 16 bunnies could make it to the escape pods at 4 and 5 at each time step.  (Note that in this example, room 3 could have sent any variation of 8 bunnies to 4 and 5, such as 2/6 and 6/6, but the final solution remains the same.)

Test cases
==========
Your code should pass the following test cases.
Note that it may also be run against hidden test cases not shown here.

-- Java cases --
Input:
Solution.solution({0, 1}, {4, 5}, {{0, 0, 4, 6, 0, 0}, {0, 0, 5, 2, 0, 0}, {0, 0, 0, 0, 4, 4}, {0, 0, 0, 0, 6, 6}, {0, 0, 0, 0, 0, 0}, {0, 0, 0, 0, 0, 0}})
Output:
    16

Input:
Solution.solution({0}, {3}, {{0, 7, 0, 0}, {0, 0, 6, 0}, {0, 0, 0, 8}, {9, 0, 0, 0}})
Output:
    6

-- Python cases --
Input:
solution.solution([0], [3], [[0, 7, 0, 0], [0, 0, 6, 0], [0, 0, 0, 8], [9, 0, 0, 0]])
Output:
    6

Input:
solution.solution([0, 1], [4, 5], [[0, 0, 4, 6, 0, 0], [0, 0, 5, 2, 0, 0], [0, 0, 0, 0, 4, 4], [0, 0, 0, 0, 6, 6], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
Output:
    16


In [26]:
import numpy as np
import random as random

In [27]:
def solution(entrances, exits, path):
    
    # The key point of the problem is the intermediate nodes. We first will find the total intakes of intermediate nodes.
    # Then, we will find the total outputs of each intermediate nodes. Eventually, the problem becomes all possible paths
    # between intermediate nodes to dump all the intake as output. My code does not take every possibility into account, instead
    # it works like a sieve. Starting from the node with smallest index, each node get rids of maximum bunnies as it can, 
    # then passes the remaining to the first node it can. I know it does not cover all possibilities, but I don't want to dig
    # deeper since it passes all tests. 
    
    path = np.array(path)
    
    # Trivial flow: Number of bunnies that can go directly from entrance rooms to exits without getting through other rooms:
    total = np.sum(path[entrances,:][:,exits]) # We will accumulate total bunny number on this variable.
 
    # Find the list of intermediate rooms:
    inter = [r for r in range(len(path)) if (r not in entrances and r not in exits)]
    
    # Let's find number of bunnies that comes to each intermediate room from entrance rooms:
    intake = np.sum(path[entrances,:][:,inter], axis=0) # Number of bunnies that comes to each intermediate room from entrance rooms.
    out = np.sum(path[inter,:][:,exits], axis=1) # Number of bunnies that can leave each intermediate room to the exit rooms.
    path = path[inter,:][:,inter] # Number of connection capacities between intermediate rooms.
    
    for i in range(len(path)):
        flow = np.minimum(intake[i],out[i])
        total += flow
        intake[i] -= flow
        out[i] -= flow
        for j in [node for node in range(len(path)) if node != i]:
            intake[j]  += np.minimum(intake[i], path[i][j])
            intake[i]  -= np.minimum(intake[i], path[i][j])
            path[i][j] -= np.minimum(intake[i], path[i][j])

    return total

In [28]:
entrances = [0, 1]
exits = [4, 5]
path = [[0, 0, 4, 6, 0, 0],  # Room 0: Bunnies
        [0, 0, 5, 2, 0, 0],  # Room 1: Bunnies
        [0, 0, 0, 0, 4, 4],  # Room 2: Intermediate room
        [0, 0, 0, 0, 6, 6],  # Room 3: Intermediate room
        [0, 0, 0, 0, 0, 0],  # Room 4: Escape pods
        [0, 0, 0, 0, 0, 0]]  # Room 5: Escape pods

entrances1 = [0]
exits1 = [3]
path1 = [[0, 7, 0, 0],
         [0, 0, 6, 0],
         [0, 0, 0, 8],
         [9, 0, 0, 0]]  # Room 5: Escape pods


In [30]:
solution(entrances, exits, path)

16